In [1]:
import os

def build_depth_map(super_path):
    depth_map_path = {}

    # Walk through the directory tree
    for root, _, files in os.walk(super_path):
        for filename in files:
            if filename.endswith('.png'):
                full_path = os.path.join(root, filename)
                # Remove super_path from the full path to get the relative path
                relative_path = os.path.relpath(full_path, super_path)
                # Split the relative path into parts (folders)
                parts = relative_path.split(os.sep)
                parts[-1]=parts[-1][len('result_merged_depth_'):-len('.png')]
                
                # Initialize nested dictionaries as needed
                current_dict = depth_map_path
                for part in parts[:-1]:  # Iterate over all parts except the last one (filename)
                    current_dict = current_dict.setdefault(part, {})
                
                # Assign the full path to the deepest nested dictionary
                current_dict[parts[-1]] = full_path

    return depth_map_path

def build_depth_map_confidence(super_path):
    depth_map_path = {}

    # Walk through the directory tree
    for root, _, files in os.walk(super_path):
        for filename in files:
            if filename.endswith('.exr'):
                full_path = os.path.join(root, filename)
                # Remove super_path from the full path to get the relative path
                relative_path = os.path.relpath(full_path, super_path)
                # Split the relative path into parts (folders)
                parts = relative_path.split(os.sep)
                parts[-1]=parts[-1][len('result_merged_conf_'):-len('.exr')]
                
                # Initialize nested dictionaries as needed
                current_dict = depth_map_path
                for part in parts[:-1]:  # Iterate over all parts except the last one (filename)
                    current_dict = current_dict.setdefault(part, {})
                
                # Assign the full path to the deepest nested dictionary
                current_dict[parts[-1]] = full_path

    return depth_map_path



In [2]:
import os

def find_train_folders(directory):
    train_folders = []
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            if dir.startswith('train'):
                train_folders.append(os.path.join(root, dir))
    return train_folders

train_path=find_train_folders('/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/')

depth_map_path={}
depth_map_path_temp={}

for path in train_path:
    depth_map_path_temp=build_depth_map(path+"/merged_depth")
    depth_map_path.update(depth_map_path_temp)

depth_map_confidence_path={}
depth_map_confidence_path_temp={}

for path in train_path:
    depth_map_confidence_path_temp=build_depth_map_confidence(path+"/merged_conf")
    depth_map_confidence_path.update(depth_map_confidence_path_temp)




import pprint
pprint.pprint(depth_map_path)
pprint.pprint(depth_map_confidence_path)


{'apt1_0': {'bottom': '/mnt/Velocity '
                      'Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_0/result_merged_depth_bottom.png',
            'center': '/mnt/Velocity '
                      'Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_0/result_merged_depth_center.png',
            'left': '/mnt/Velocity '
                    'Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_0/result_merged_depth_left.png',
            'right': '/mnt/Velocity '
                     'Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_0/result_merged_depth_right.png',
            'top': '/mnt/Velocity '
                   'Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_0/result_merged_depth_top.png'},
 'apt1_1': {'bottom': '/mnt/Velocity '
                      'Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_1/result_merged_depth_bottom.png',
   

In [3]:
import numpy as np
import cv2
import OpenEXR
import Imath

def weighted_depth_map(depth_map, confidence_map, confidence_threshold=0.95):
    # Flatten the depth map and confidence map
    depth_values = depth_map.flatten()
    confidence_values = confidence_map.flatten()
    
    # Filter out low confidence values if a threshold is provided
    if confidence_threshold > 0.0:
        mask = confidence_values >= confidence_threshold
        depth_values = depth_values[mask]
    
    return depth_values

def load_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    r = np.frombuffer(exr_file.channel('R', pt), dtype=np.float32)
    r.shape = (height, width)
    return r

def predict_focal_length(depth_map_path,depth_confidence_path):
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    depth_map_confidence = load_exr(depth_confidence_path)

    # Define max and min values
    max_depth = 100.0
    min_depth = 0.2


    # Convert the depth map to meters using the given formula
    depth_values = weighted_depth_map(depth_map,depth_map_confidence)

    depth_map_in_meters = (max_depth * min_depth) / (max_depth - (max_depth - min_depth) * (depth_values / 255.0))

    # Compute the median value in the entire depth map
    median_depth = np.median(depth_map_in_meters)

    final_focus=median_depth*1000

    return final_focus

slice_focal_length=[3910.92,2289.27,1508.71,1185.83,935.91,801.09,700.37,605.39,546.23,486.87,447.99,407.40,379.91,350.41,329.95,307.54,291.72,274.13,261.53,247.35,237.08,225.41,216.88,207.10,198.18,191.60,183.96,178.29,171.69,165.57,160.99,155.61,150.59,146.81,142.35,138.98,134.99,131.23,127.69,124.99,121.77,118.73,116.40,113.63,110.99,108.47,106.54,104.23,102.01]

def find_closest(value, num_list):
    closest_value = min(num_list, key=lambda x: abs(x - value))
    return closest_value

def predict_slice(depth_map_path,depth_confidence_path):
    predicted_focus=predict_focal_length(depth_map_path,depth_confidence_path)
    closest_value = find_closest(predicted_focus, slice_focal_length)
    true_slice=slice_focal_length.index(closest_value)
    return true_slice

In [5]:
ground_truth=depth_map_path.copy()

for image_type in ground_truth:
    for pos in ground_truth[image_type]:
        ground_truth[image_type][pos]=predict_slice(depth_map_path[image_type][pos],depth_map_confidence_path[image_type][pos])

import pprint
pprint.pprint(ground_truth)

{'apt1_0': {'bottom': 7, 'center': 6, 'left': 6, 'right': 6, 'top': 5},
 'apt1_1': {'bottom': 2, 'center': 2, 'left': 2, 'right': 2, 'top': 2},
 'apt1_2': {'bottom': 0, 'center': 1, 'left': 1, 'right': 1, 'top': 1},
 'apt1_3': {'bottom': 4, 'center': 4, 'left': 4, 'right': 4, 'top': 4},
 'apt1_4': {'bottom': 4, 'center': 4, 'left': 4, 'right': 4, 'top': 4},
 'apt1_5': {'bottom': 3, 'center': 3, 'left': 3, 'right': 3, 'top': 3},
 'apt1_6': {'bottom': 1, 'center': 1, 'left': 1, 'right': 1, 'top': 1},
 'apt1_7': {'bottom': 1, 'center': 3, 'left': 1, 'right': 1, 'top': 1},
 'apt1_8': {'bottom': 6, 'center': 6, 'left': 6, 'right': 6, 'top': 3},
 'apt1_9': {'bottom': 6, 'center': 6, 'left': 6, 'right': 5, 'top': 5},
 'apt2_0': {'bottom': 10, 'center': 10, 'left': 10, 'right': 9, 'top': 8},
 'apt2_1': {'bottom': 1, 'center': 1, 'left': 1, 'right': 1, 'top': 1},
 'apt2_2': {'bottom': 7, 'center': 7, 'left': 7, 'right': 7, 'top': 6},
 'apt2_3': {'bottom': 11, 'center': 9, 'left': 11, 'right': 9

In [ ]:
def ground_truth_to_dataset(truth):
    labels=[]
    count=0
    for image_type in truth:
        if count==len(labels):
            labels.append([])
            labels.append([])
            labels.append([])
            labels.append([])
            labels.append([])
        for 